In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [2]:
# Script to visualize demonstrations according to the timestamps
# First get the first timestamp of the xela sensor
# Then with given fps given get image frames - find closest tactile and image observations

In [3]:
from tactile_learning.datasets.preprocess import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [4]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, 'cam_0_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [5]:

def dump_states(root, tactile_indices, image_indices):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_0_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [12]:
# root = '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_box_opening_1'
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_box_opening_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_hammer_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_book_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_bottle_opening_1',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_reverse_peg_4',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_new_joystick_2'
# ]
roots = [
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_handle_grabbing_1',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_bowl_handle_lifting_4',
    # '/home/irmak/Workspace/Holo-Bot/extracted_data/both_robots/demonstration_bowl_handle_lifting_6',
    # '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_4cm_forward_start',
    # '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_10cm_forward_start',
    # '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_10cm_left_start',
    # '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_10cm_right_start',
    '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_5cm_up_start'
]

retrieval_fps = 2
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices)


tactile_indices: [0, 50, 99, 149, 196, 247, 295, 345, 395, 443, 491, 541, 591, 639, 690, 737, 788, 835, 884, 933, 982, 1031, 1080, 1129, 1177, 1227, 1276, 1326, 1374, 1424, 1472, 1522, 1569, 1619, 1667, 1717, 1766, 1815, 1864, 1914, 1962, 2012, 2060, 2110, 2158, 2208, 2257, 2307, 2354, 2405, 2454, 2504, 2551, 2602, 2649, 2697, 2746, 2795, 2844, 2893, 2942, 2991, 3041, 3089, 3136, 3188, 3235, 3285, 3333, 3383, 3432, 3481, 3530, 3580, 3628, 3677, 3726, 3775, 3824, 3874, 3921, 3971, 4019, 4069, 4117, 4168, 4215, 4265, 4313, 4363, 4414, 4461, 4509, 4560, 4608, 4658, 4706, 4756, 4805, 4854, 4902, 4951, 4999, 5048, 5097, 5145, 5196, 5243, 5293, 5342, 5391, 5439, 5489, 5536, 5586, 5634, 5681, 5731, 5781, 5829, 5879, 5928, 5976, 6026, 6073, 6124, 6172, 6227, 6270, 6321, 6368, 6416, 6466, 6514, 6564, 6612, 6662, 6710, 6760, 6809, 6859, 6907, 6956, 7005, 7054, 7103, 7153, 7201, 7250, 7299, 7347, 7396, 7446, 7494, 7544, 7591, 7642, 7690, 7740, 7788, 7838, 7886, 7937, 7984, 8034, 8082, 8131, 8182,

100%|██████████| 306/306 [05:51<00:00,  1.15s/it]


tactile_indices: [0, 52, 100, 150, 198, 247, 296, 345, 394, 444, 492, 543, 592, 641, 689, 737, 787, 836, 886, 934, 983, 1031, 1082, 1129, 1180, 1227, 1277, 1329, 1377, 1424, 1474, 1522, 1571, 1621, 1670, 1719, 1767, 1817, 1865, 1915, 1963, 2012, 2063, 2111, 2159, 2208, 2257, 2307, 2355, 2404, 2452, 2502, 2551, 2598, 2648, 2697, 2747, 2797, 2846, 2894, 2943, 2993, 3042, 3092, 3139, 3190, 3238, 3288, 3336, 3387, 3435, 3483, 3534, 3581, 3631, 3679, 3730, 3778, 3827, 3875, 3925, 3973, 4024, 4071, 4121, 4169, 4220, 4269, 4318, 4367, 4417, 4465, 4515, 4564, 4613, 4662, 4710, 4759, 4807, 4857, 4906, 4955, 5004, 5055, 5102, 5152, 5200, 5250, 5299, 5348, 5397, 5446, 5494, 5544, 5593, 5641, 5691, 5740, 5788, 5839, 5887, 5936, 5985, 6036, 6083, 6133, 6181, 6235, 6279, 6328, 6377, 6427, 6475, 6525, 6574, 6623, 6672, 6721, 6768, 6820, 6868, 6917, 6967, 7016, 7065, 7115, 7163, 7213, 7261, 7310, 7359, 7409, 7457, 7507, 7556, 7606, 7654, 7704, 7752, 7802, 7850, 7900, 7951, 7999, 8046, 8097, 8145, 8193

100%|██████████| 216/216 [04:10<00:00,  1.16s/it]


tactile_indices: [0, 52, 99, 148, 197, 246, 296, 346, 393, 442, 491, 539, 588, 637, 687, 734, 784, 833, 883, 931, 982, 1029, 1079, 1127, 1177, 1224, 1275, 1323, 1373, 1420, 1471, 1518, 1567, 1621, 1666, 1715, 1764, 1813, 1863, 1912, 1961, 2010, 2060, 2107, 2157, 2205, 2255, 2304, 2353, 2402, 2452, 2502, 2549, 2600, 2648, 2698, 2745, 2795, 2843, 2891, 2940, 2989, 3039, 3088, 3136, 3185, 3232, 3283, 3331, 3380, 3429, 3479, 3528, 3577, 3626, 3675, 3725, 3774, 3822, 3872, 3920, 3969, 4017, 4066, 4116, 4165, 4215, 4263, 4313, 4361, 4411, 4460, 4510, 4558, 4609, 4656, 4706, 4754, 4805, 4852, 4903, 4950], image_indices: [70, 85, 100, 114, 129, 144, 159, 173, 188, 203, 218, 233, 248, 263, 278, 293, 306, 321, 336, 351, 366, 381, 396, 410, 425, 440, 455, 470, 484, 499, 514, 529, 543, 558, 573, 587, 602, 617, 632, 647, 662, 676, 691, 706, 720, 735, 749, 764, 779, 794, 809, 824, 838, 852, 867, 881, 896, 910, 925, 940, 954, 969, 984, 999, 1013, 1028, 1042, 1057, 1072, 1086, 1101, 1116, 1130, 1144, 

100%|██████████| 102/102 [02:03<00:00,  1.21s/it]


In [13]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, 'visualization.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_10cm_forward_start


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_10cm_left_start


[libx264 @ 0x5587fe75fd80] using SAR=1/1
[libx264 @ 0x5587fe75fd80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5587fe75fd80] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x5587fe75fd80] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demo

dumping video in root: /home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demonstration_10cm_right_start


[libx264 @ 0x556f5aaf3a00] using SAR=1/1
[libx264 @ 0x556f5aaf3a00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x556f5aaf3a00] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x556f5aaf3a00] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/deployment_data/box_handle_lifting/demo

In [ ]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
